In [1280]:
import os
import json
import numpy as np
import tensorflow as tf

In [1281]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [1282]:
# Name of directory for grids
directory_name = './data/mid/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    try:
        data = json.load(f)
    except Exception:
        continue

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        if len(i['gridworld']) != 625:
            print(file_name)
            continue
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()

# direct_count = [0 for i in range(4)]
# sample = []

# # iterate through all training grids
# for file_name in os.listdir(directory_name):
#     f = open(directory_name + file_name)
#     data = json.load(f)

#     # Iterate through all the data in a given grid and append their input and output values
#     for i in data:
#         if direct_count[i['direction']] < 8000:
#             sample.append(i)
#             direct_count[i['direction']] += 1
    
#     # Close file socket
#     f.close()

# np.random.shuffle(sample)
# for i in sample:
#     x_train_grid.append(i['gridworld'])
#     x_train_position.append(i['position'])
#     x_train_distance.append(i['distance'])
#     x_train_locals.append(i['local'])
#     y_train.append(i['direction'])
    

In [1283]:
#print(np.reshape(x_train_grid[25], (30,30)))
#print(x_train_position[25])
#print(y_train[25])

In [1284]:
print(direct_count)

[2240, 5706, 12608, 16074]


In [1285]:
# Name of directory for grids
directory_name = './data/midtest/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [1286]:
#print(x_train_position)
#print(y_train)

In [1287]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 25, 25) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1) )
train_in_locals = np.reshape( x_train_locals, (-1, 5, 5) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 25, 25) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1) )
test_in_locals = np.reshape( x_test_locals, (-1, 5, 5) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [1288]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (25,25) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [1289]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [1290]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (5,5) )
flatten_local = tf.keras.layers.Flatten()( local_input )

In [1291]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_local, flatten_position])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 652), dtype=tf.float32, name=None), name='concatenate_67/concat:0', description="created by layer 'concatenate_67'")


In [1292]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 40, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 20, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [1293]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, local_input, position_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [1294]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [1295]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

84	119	0	19
163	319	0	71
270	824	1	135
354	830	0	377


In [1296]:
# Train the model
history = model.fit( [train_in_grid, train_in_locals, train_in_position], train_out, validation_data=([test_in_grid, test_in_locals, test_in_position], test_out), epochs = 7 )

Epoch 1/7
1145/1145 [==============================] - 5s 4ms/step - loss: 0.2143 - accuracy: 0.9271 - val_loss: 0.1128 - val_accuracy: 0.9616
Epoch 2/7
1145/1145 [==============================] - 4s 4ms/step - loss: 0.0997 - accuracy: 0.9650 - val_loss: 0.0845 - val_accuracy: 0.9686
Epoch 3/7
1145/1145 [==============================] - 4s 4ms/step - loss: 0.0823 - accuracy: 0.9703 - val_loss: 0.0842 - val_accuracy: 0.9697
Epoch 4/7
1145/1145 [==============================] - 4s 4ms/step - loss: 0.0700 - accuracy: 0.9754 - val_loss: 0.0728 - val_accuracy: 0.9720
Epoch 5/7
1145/1145 [==============================] - 4s 3ms/step - loss: 0.0619 - accuracy: 0.9781 - val_loss: 0.0657 - val_accuracy: 0.9764
Epoch 6/7
1145/1145 [==============================] - 4s 3ms/step - loss: 0.0559 - accuracy: 0.9810 - val_loss: 0.0613 - val_accuracy: 0.9790
Epoch 7/7
1145/1145 [==============================] - 4s 4ms/step - loss: 0.0500 - accuracy: 0.9830 - val_loss: 0.0636 - val_accuracy: 0.9767

In [1297]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

181	12	16	13
5	535	6	7
3	5	1221	1
5	6	4	1546


In [1298]:
model.save('./models/agent1_NN')

INFO:tensorflow:Assets written to: ./models/agent1_NN/assets
